In [ ]:
# fundamental_data_extractor.py
import pandas as pd
import os
import logging
from tqdm import tqdm
import fundamentus

In [ ]:
# Configuração do logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

# Diretório para salvar os dados
DATA_DIR = "data"
FUNDAMENTAL_DIR = os.path.join(DATA_DIR, "fundamental")
os.makedirs(FUNDAMENTAL_DIR, exist_ok=True)

In [ ]:
def load_tickers_from_csv(file_path: str) -> list:
    """Carrega a lista de tickers a partir de um arquivo CSV."""
    if not os.path.exists(file_path):
        logging.error(f"Arquivo de tickers não encontrado em: {file_path}")
        return []
    
    df = pd.read_csv(file_path)
    
    # Heurística para encontrar a coluna de tickers
    if 'Ticker' in df.columns:
        tickers = df['Ticker'].dropna().astype(str).tolist()
    else:
        tickers = df.iloc[:, 0].dropna().astype(str).tolist()
        
    # Normaliza para o formato que `fundamentus` espera (ex: 'PETR4' sem '.SA')
    tickers_normalized = [t.replace('.SA', '') for t in tickers]
    logging.info(f"{len(tickers_normalized)} tickers carregados de {file_path}")
    return tickers_normalized

In [ ]:
# fundamental_data_extractor.py -> Função get_fundamental_data atualizada
def get_fundamental_data(tickers: list) -> pd.DataFrame:
    """
    Busca dados fundamentalistas para uma lista de tickers usando a biblioteca fundamentus.
    Inclui uma etapa de sanitização para lidar com valores que são listas ou Series.
    Retorna um único DataFrame com todos os dados.
    """
    if not tickers:
        logging.warning("A lista de tickers está vazia. Nenhum dado será buscado.")
        return pd.DataFrame()

    all_data = []
    
    logging.info("Iniciando a captura de dados fundamentalistas...")
    for ticker in tqdm(tickers, desc="Buscando dados fundamentalistas"):
        try:
            data = fundamentus.get_detalhes_papel(ticker)
            data['ticker'] = ticker
            all_data.append(data)
        except Exception as e:
            logging.warning(f"Não foi possível obter dados para o ticker {ticker}. Erro: {e}")
            continue
            
    if not all_data:
        logging.error("Nenhum dado fundamentalista foi capturado com sucesso.")
        return pd.DataFrame()

    # --- INÍCIO DA CORREÇÃO (AGORA MAIS ROBUSTA) ---
    # Etapa de Sanitização: Converte valores que são listas, tuplas ou Series em strings.
    sanitized_data = []
    for company_data in all_data:
        sanitized_item = {}
        for key, value in company_data.items():
            if isinstance(value, (list, tuple)):
                # Se o valor for uma lista, une seus elementos em uma única string
                sanitized_item[key] = ' / '.join(map(str, value))
            
            # ADICIONADO: Verifica também se o valor é uma Series do pandas
            elif isinstance(value, pd.Series):
                # Converte a Series para uma lista de strings e depois une
                sanitized_item[key] = ' / '.join(value.astype(str).tolist())
                
            else:
                # Caso contrário, mantém o valor original
                sanitized_item[key] = value
        sanitized_data.append(sanitized_item)
    # --- FIM DA CORREÇÃO ---

    # Converte a lista SANITIZADA de dicionários em um DataFrame
    df_fundamental = pd.DataFrame(sanitized_data)
    
    # Reordena as colunas para ter 'ticker' primeiro
    if 'ticker' in df_fundamental.columns:
        cols = ['ticker'] + [col for col in df_fundamental.columns if col != 'ticker']
        df_fundamental = df_fundamental[cols]
        
    return df_fundamental

if __name__ == "__main__":
    # 1. Carregar os tickers do arquivo CSV gerado anteriormente
    tickers_csv_path = "tickers_ibrx100_full.csv"
    ibrx_tickers = load_tickers_from_csv(tickers_csv_path)
    
    # 2. Buscar os dados fundamentalistas
    df_fund = get_fundamental_data(ibrx_tickers)
    
    # 3. Salvar os resultados
    if not df_fund.empty:
        output_path_parquet = os.path.join(FUNDAMENTAL_DIR, "fundamentus_data.parquet")
        output_path_csv = os.path.join(FUNDAMENTAL_DIR, "fundamentus_data.csv")
        
        # Salva em Parquet (preferencial)
        df_fund.to_parquet(output_path_parquet, index=False)
        logging.info(f"Dados fundamentalistas salvos em: {output_path_parquet}")
        
        # Salva também em CSV para fácil visualização
        df_fund.to_csv(output_path_csv, index=False)
        logging.info(f"Dados fundamentalistas salvos também em: {output_path_csv}")
        
        print("\n--- Amostra dos Dados Coletados ---")
        print(df_fund.head())
        print(f"\nDimensões do DataFrame: {df_fund.shape}")